## Plot composites GEV ##

Plot the composites of temperature and wind speed for the worst demand net wind days.

In [1]:
# Imports
import os
import sys
import glob
import time
import re

# Third-party libraries
import numpy as np
import pandas as pd
import xarray as xr
import iris
import iris.coords
import cftime
import matplotlib.pyplot as plt
import matplotlib.ticker as mplticker
import cartopy.crs as ccrs

# Specific third-party imports
from matplotlib import colors
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.ticker import FuncFormatter
from tqdm import tqdm
from scipy.stats import pearsonr
from iris.util import equalise_attributes

# Import types
from typing import Any, Callable, Union, List, Tuple

# Local imports
from plotting_functions import format_func, format_func_one_decimal

# Non-local imports
sys.path.append("/home/users/benhutch/unseen_functions/")
from functions import plot_dps_composite, plot_daily_field

/home/users/benhutch/.conda/envs/bens-conda-env2/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
# ---------------------------------
# Load the dataframe containing the dates for the ERA5 period
# ---------------------------------

# Set up the path to the obs dnw dataframe
obs_dnw_path = "/home/users/benhutch/unseen_multi_year/dfs/UK_obs_demand_net_wind_max_djf_1960-2017_2025-03-18.csv"

# Load the obs dnw dataframe
obs_dnw_df = pd.read_csv(obs_dnw_path)

In [3]:
%%time
# ---------------------------------
# Load in the ERA5 daily temperature data
# ---------------------------------

# set up the data path to the temp data
temp_path = "/gws/nopw/j04/canari/users/benhutch/ERA5/ERA5_t2m_daily_1950_2020.nc"

# Load the data
temp_cube = iris.load_cube(temp_path, "t2m")

CPU times: user 29.8 ms, sys: 30.6 ms, total: 60.4 ms
Wall time: 19.9 s


In [4]:
%%time
# ---------------------------------
# Load in the ERA5 daily 10m wind speed data
# ---------------------------------

# Set up the base path
base_path = "/gws/nopw/j04/canari/users/benhutch/ERA5/"

# Set up the path to the wind data
obs_path = os.path.join(base_path, "ERA5_wind_daily_1952_2020.nc")

# Load the data
wind_cube = iris.load_cube(obs_path, "si10")

CPU times: user 6.28 ms, sys: 6.9 ms, total: 13.2 ms
Wall time: 347 ms


In [5]:
# ---------------------------------
# Restrict both obs datasets to a specific Europe box
# ---------------------------------

# Define the box to constrain to
test_europe_box = {
    "lon1": -11,  # degrees east
    "lon2": 30,
    "lat1": 35,  # degrees north
    "lat2": 70,
}

# Define the constraints
constraints = iris.Constraint(
    longitude=lambda cell: test_europe_box["lon1"] <= cell <= test_europe_box["lon2"],
    latitude=lambda cell: test_europe_box["lat1"] <= cell <= test_europe_box["lat2"],
)

# Apply the constraints
temp_cube_eu = temp_cube.extract(constraints)
wind_cube_eu = wind_cube.extract(constraints)

In [7]:
temp_cube_eu

<iris 'Cube' of 2 metre temperature / (K) (time: 25933; latitude: 124; longitude: 146)>

In [8]:
# ---------------------------------
# Restrict the obs data to specific dates
# ---------------------------------

# Define the start and end dates
start_date = "1960-12-01"
end_date = "2017-03-01"

# Convert the dates to datetime objects
start_date_dt = pd.to_datetime(start_date)
end_date_dt = pd.to_datetime(end_date)

# Create a time constraint
time_constraint = iris.Constraint(time=lambda cell: start_date_dt <= cell.point < end_date_dt)

# Apply the date constraints
temp_cube_eu = temp_cube_eu.extract(time_constraint)

# Same for the wind data
wind_cube_eu = wind_cube_eu.extract(time_constraint)

In [ ]:
# ---------------------------------
# Calculate the climatology for the observed datasets
# ---------------------------------

# Calculate the climatology for temp data
